In [1]:
%matplotlib inline
import matplotlib as mlp
mlp.rcParams['figure.dpi'] = 150

# diasbling warning messages
import warnings
warnings.filterwarnings("ignore")

# notebook dependencies
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

#visualizations
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = "darkgrid")

# math modules
from math import sqrt
import scipy.stats as stats

import acquire
import prepare

# sklearn data science library
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import mean_squared_error
from scipy.stats.mstats import winsorize

from sklearn.impute import IterativeImputer
from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.decomposition import PCA 
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

In [2]:
# acquire the initial dataset
df = acquire.get_bach_df()

# clean and prep initial dataset
df = prepare.clean_college_df(df)
df = prepare.clean_high_percentage_nulls(df)

# pull-in target variables
# ensure you have `2017_2018_2019_earning_by_major.csv` within working folder
df = prepare.obtain_target_variables(df)

df.head()

dataframe shape: (71901, 139)
dataframe shape: (69012, 117)
dataframe shape: (68546, 127)


,unit_id_institution,college_name,institution_control,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,degree_name,degree_code,major_code,major_name,branch_number,avg_net_price_public,avg_net_price_private,pub_fam_income_0_30000,private_fam_income_0_30000,program_fam_income_0_30000,other_fam_income_0_30000,pub_fam_income_30001_48000,private_fam_income_30001_48000,program_fam_income_30001_48000,other_fam_income_30001_48000,pub_fam_income_48001_75000,private_fam_income_48001_75000,program_fam_income_48001_75000,other_fam_income_48001_75000,pub_fam_income_75001_110000,private_fam_income_75001_110000,program_fam_income_75001_110000,other_fam_income_75001_110000,pub_fam_income_over_110000,private_fam_income_over_110000,program_fam_income_over_110000,other_fam_income_over_110000,full_time_net_tuition_revenue,off_campus_cost_of_attendace,on_campus_cost_of_attendace,admission_rate,graduate_number,ACT_score_mid,avg_sat_admitted,required_score,avg_faculty_salary,online_only,comp_rt_ft_150over_expected_time,comp_rt_ft_150over_expected_time_two_races,comp_rt_ft_150over_expected_time_native_american,comp_rt_ft_150over_expected_time_asian,comp_rt_ft_150over_expected_time_black,comp_rt_ft_150over_expected_time_hispanic,comp_rt_ft_150over_expected_time_non_resident,comp_rt_ft_150over_expected_time_unknown_race,comp_rt_ft_150over_expected_time_white,share_entering_students_first_ft,share_of_part_time,first_time_ft_student_retention,first_time_pt_student_retention,enrollment_share_two_races,enrollment_share_native_american,enrollment_share_asian,enrollment_share_black,enrollment_share_hispanic,enrollment_share_pac_islander,enrollment_share_non_resident,enrollment_share_unknown,enrollment_share_white,undergraduate_number_pell_grant_fedral_loan,med_debt_pell_students,median_debt_completed,median_debt_non_first_generation,median_debt_non_pell,fedral_loan_full_time_first_time_undergraduate,pell_grant_full_time_first_time_undergraduate,deg_percent_awarded_agriculture_operations,deg_percent_awarded_natural_resources,deg_percent_awarded_architecture,deg_percent_awarded_area_ethnic_cultural_gender,deg_percent_awarded_communication_journalism,deg_percent_awarded_communication_tech,deg_percent_awarded_computer_science,deg_percent_awarded_personal_culinary_services,deg_percent_awarded_education,deg_percent_awarded_engineering,deg_percent_awarded_engineering_tech,deg_percent_awarded_foreign_language_literatures,deg_percent_awarded_human_science,deg_percent_awarded_legal_profession,deg_percent_awarded_english_lang,deg_percent_awarded_general_studies,deg_percent_awarded_library_sciences,deg_percent_awarded_bio_sciences,deg_percent_awarded_mathematics,deg_percent_awarded_military_tech,deg_percent_awarded_intedisciplinary_studies,deg_percent_awarded_leisure_fitness,deg_percent_awarded_philosophy,deg_percent_awarded_theology,deg_percent_awarded_physical_sciences,deg_percent_awarded_science_tech,deg_percent_awarded_psychology,deg_percent_awarded_homeland_security,deg_percent_awarded_public_admin,deg_percent_awarded_social_sciences,deg_percent_awarded_construction_trades,deg_percent_awarded_mechanic_repair,deg_percent_awarded_precision_production,deg_percent_awarded_transportation_materials,deg_percent_awarded_visual_and_performing_arts,deg_percent_awarded_health,deg_percent_awarded_business_management,deg_percent_awarded_history,non_deg_seeking,avg_net_price,major_category,2017,2018,2019,Grand Total,roi_5yr,pct_roi_5yr,roi_10yr,pct_roi_10yr,roi_20yr,pct_roi_20yr
0,100654.0,Alabama A & M University,Public,AL,35762,Normal,5.0,1.0,3.0,3.0,Bachelor’s Degree,3,100,"Agriculture, General.",1.0,14444.0,0.0,375.0,NaN,NaN,NaN,113.0,NaN,NaN,NaN,61.0,NaN,NaN,NaN,27.0,NaN,NaN,NaN,16.0,NaN,NaN,NaN,7413.0,9128.0,9128.0,0.8986,963.0,18.0,957.0,1.0,7101.0,0.0,0.2685,0.2500,NaN,NaN,0.2681,0.25,NaN,0.3750,0.2500,0.8987,0.0587,0.6087,1.0000,0.0118,0.0022,0.0018,0.9120,0.0088,0.0016,0.007,0.0361,0.0186,5039.0,17500,33375,16421,10250,0.714

In [3]:
# collapse/create single family income columns 

df = prepare.create_fam_income_columns(df)

dataframe shape: (68546, 112)


In [4]:
# checking percentage of remaining nulls across features
null_percentages = df.isnull().mean().round(2)

with pd.option_context('display.max_rows', None):
    print(null_percentages)

unit_id_institution                                 0.00
college_name                                        0.00
institution_control                                 0.00
state_post_code                                     0.00
zip_code                                            0.00
city                                                0.00
region_ipeds                                        0.00
title_IV_eligibility                                0.00
pred_degree                                         0.00
pred_degree_0and4                                   0.00
degree_name                                         0.00
degree_code                                         0.00
major_code                                          0.00
major_name                                          0.00
branch_number                                       0.00
avg_net_price_public                                0.00
avg_net_price_private                               0.00
full_time_net_tuition_revenue  

In [5]:
# splitting the data

train, validate, test = prepare.split_data(df)

train shape: (38385, 112)
validate shape: (16451, 112)
test shape: (13710, 112)


In [6]:
# capping training df

train = prepare.percentile_capping(train, 0.1, 0.1)
train.shape

(38385, 112)

In [7]:
# capping validate and test dfs

validate = prepare.percentile_capping(validate, 0.1, 0.1)
test = prepare.percentile_capping(test, 0.1, 0.1)

In [8]:
validate.shape

(16451, 112)

In [9]:
test.shape
test.head()

,unit_id_institution,college_name,institution_control,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,degree_name,degree_code,major_code,major_name,branch_number,avg_net_price_public,avg_net_price_private,full_time_net_tuition_revenue,off_campus_cost_of_attendace,on_campus_cost_of_attendace,admission_rate,graduate_number,ACT_score_mid,avg_sat_admitted,required_score,avg_faculty_salary,online_only,comp_rt_ft_150over_expected_time,comp_rt_ft_150over_expected_time_two_races,comp_rt_ft_150over_expected_time_native_american,comp_rt_ft_150over_expected_time_asian,comp_rt_ft_150over_expected_time_black,comp_rt_ft_150over_expected_time_hispanic,comp_rt_ft_150over_expected_time_non_resident,comp_rt_ft_150over_expected_time_unknown_race,comp_rt_ft_150over_expected_time_white,share_entering_students_first_ft,share_of_part_time,first_time_ft_student_retention,first_time_pt_student_retention,enrollment_share_two_races,enrollment_share_native_american,enrollment_share_asian,enrollment_share_black,enrollment_share_hispanic,enrollment_share_pac_islander,enrollment_share_non_resident,enrollment_share_unknown,enrollment_share_white,undergraduate_number_pell_grant_fedral_loan,med_debt_pell_students,median_debt_completed,median_debt_non_first_generation,median_debt_non_pell,fedral_loan_full_time_first_time_undergraduate,pell_grant_full_time_first_time_undergraduate,deg_percent_awarded_agriculture_operations,deg_percent_awarded_natural_resources,deg_percent_awarded_architecture,deg_percent_awarded_area_ethnic_cultural_gender,deg_percent_awarded_communication_journalism,deg_percent_awarded_communication_tech,deg_percent_awarded_computer_science,deg_percent_awarded_personal_culinary_services,deg_percent_awarded_education,deg_percent_awarded_engineering,deg_percent_awarded_engineering_tech,deg_percent_awarded_foreign_language_literatures,deg_percent_awarded_human_science,deg_percent_awarded_legal_profession,deg_percent_awarded_english_lang,deg_percent_awarded_general_studies,deg_percent_awarded_library_sciences,deg_percent_awarded_bio_sciences,deg_percent_awarded_mathematics,deg_percent_awarded_military_tech,deg_percent_awarded_intedisciplinary_studies,deg_percent_awarded_leisure_fitness,deg_percent_awarded_philosophy,deg_percent_awarded_theology,deg_percent_awarded_physical_sciences,deg_percent_awarded_science_tech,deg_percent_awarded_psychology,deg_percent_awarded_homeland_security,deg_percent_awarded_public_admin,deg_percent_awarded_social_sciences,deg_percent_awarded_construction_trades,deg_percent_awarded_mechanic_repair,deg_percent_awarded_precision_production,deg_percent_awarded_transportation_materials,deg_percent_awarded_visual_and_performing_arts,deg_percent_awarded_health,deg_percent_awarded_business_management,deg_percent_awarded_history,non_deg_seeking,avg_net_price,major_category,2017,2018,2019,Grand Total,roi_5yr,pct_roi_5yr,roi_10yr,pct_roi_10yr,roi_20yr,pct_roi_20yr,income_0_30000,income_30001_48000,income_48001_75000,income_75001_110000,income_over_110000
32633,180258.0,University of Providence,"Private, nonprofit",MT,59405-4996,Great Falls,7.0,1.0,3.0,3.0,Bachelor’s Degree,3,4511,Sociology.,1.0,0.0,20355.0,10031.0,7900.0,9600.0,0.5636,162.0,24.2762,1166.2046,2.0,5795.0,0.0,0.4580,NaN,0.2000,NaN,0.3333,0.5000,0.6667,0.5000,0.4409,0.4651,0.3123,0.6140,0.4444,0.0097,0.0106,0.0342,0.0445,0.0422,0.0050,0.0205,0.0799,0.7614,930.0,17685,20710,9000,7675,0.5870,0.3913,0.000,0.0000,0.0000,0.0000,0.0000,0.0,0.0038,0.0,0.0577,0.0000,0.0,0.0000,0.0000,0.0079,0.0000,0.0269,0.0,0.0577,0.0231,0.0,0.0000,0.0308,0.0038,0.0000,0.0000,0.0,0.0308,0.0692,0.0000,0.0038,0.0,0.0,0.0,0.0,0.0000,0.3278,0.1231,0.0038,2615.0,20355.0,Social Sciences,69607.0267,70888.9152,74486.2376,214982.1795,0.5501,55.0078,2.2806,228.0644,6.3009,630.0877,24.0,13.0,13.0,13.0,11.0
47762,215655.0,Robert Morris University,"Private, nonprofit",PA,15108-1189,Moon Township,2.0,1.0,3.0,3.0,Bachelor’s Degree,3,5203,Accounting and Related S

In [10]:
# train/transform train df using iterative imputer

train_imputed = prepare.train_iterative_imputer(train)
print(f'df shape: {train_imputed.shape}')

df shape: (38385, 112)


In [11]:
# check the head 

train_imputed.head()

,unit_id_institution,college_name,institution_control,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,degree_name,degree_code,major_code,major_name,branch_number,avg_net_price_public,avg_net_price_private,full_time_net_tuition_revenue,off_campus_cost_of_attendace,on_campus_cost_of_attendace,admission_rate,graduate_number,ACT_score_mid,avg_sat_admitted,required_score,avg_faculty_salary,online_only,comp_rt_ft_150over_expected_time,comp_rt_ft_150over_expected_time_two_races,comp_rt_ft_150over_expected_time_native_american,comp_rt_ft_150over_expected_time_asian,comp_rt_ft_150over_expected_time_black,comp_rt_ft_150over_expected_time_hispanic,comp_rt_ft_150over_expected_time_non_resident,comp_rt_ft_150over_expected_time_unknown_race,comp_rt_ft_150over_expected_time_white,share_entering_students_first_ft,share_of_part_time,first_time_ft_student_retention,first_time_pt_student_retention,enrollment_share_two_races,enrollment_share_native_american,enrollment_share_asian,enrollment_share_black,enrollment_share_hispanic,enrollment_share_pac_islander,enrollment_share_non_resident,enrollment_share_unknown,enrollment_share_white,undergraduate_number_pell_grant_fedral_loan,med_debt_pell_students,median_debt_completed,median_debt_non_first_generation,median_debt_non_pell,fedral_loan_full_time_first_time_undergraduate,pell_grant_full_time_first_time_undergraduate,deg_percent_awarded_agriculture_operations,deg_percent_awarded_natural_resources,deg_percent_awarded_architecture,deg_percent_awarded_area_ethnic_cultural_gender,deg_percent_awarded_communication_journalism,deg_percent_awarded_communication_tech,deg_percent_awarded_computer_science,deg_percent_awarded_personal_culinary_services,deg_percent_awarded_education,deg_percent_awarded_engineering,deg_percent_awarded_engineering_tech,deg_percent_awarded_foreign_language_literatures,deg_percent_awarded_human_science,deg_percent_awarded_legal_profession,deg_percent_awarded_english_lang,deg_percent_awarded_general_studies,deg_percent_awarded_library_sciences,deg_percent_awarded_bio_sciences,deg_percent_awarded_mathematics,deg_percent_awarded_military_tech,deg_percent_awarded_intedisciplinary_studies,deg_percent_awarded_leisure_fitness,deg_percent_awarded_philosophy,deg_percent_awarded_theology,deg_percent_awarded_physical_sciences,deg_percent_awarded_science_tech,deg_percent_awarded_psychology,deg_percent_awarded_homeland_security,deg_percent_awarded_public_admin,deg_percent_awarded_social_sciences,deg_percent_awarded_construction_trades,deg_percent_awarded_mechanic_repair,deg_percent_awarded_precision_production,deg_percent_awarded_transportation_materials,deg_percent_awarded_visual_and_performing_arts,deg_percent_awarded_health,deg_percent_awarded_business_management,deg_percent_awarded_history,non_deg_seeking,avg_net_price,major_category,2017,2018,2019,Grand Total,roi_5yr,pct_roi_5yr,roi_10yr,pct_roi_10yr,roi_20yr,pct_roi_20yr,income_0_30000,income_30001_48000,income_48001_75000,income_75001_110000,income_over_110000
25656,168847.0,Baker College,"Private, nonprofit",MI,48867,Owosso,3.0,1.0,3.0,3.0,Bachelor’s Degree,3.0,2401.0,"Liberal Arts and Sciences, General Studies and...",2.0,0.0,12329.0,9932.0,7200.0,8029.0,0.8002,358.000000,23.5303,1142.0209,2.0,5679.0,0.0,0.3217,0.2188,0.1000,0.2857,0.1694,0.2500,0.440497,0.2222,0.3590,0.4595,0.3185,0.6154,0.438000,0.0427,0.0040,0.0151,0.1035,0.0386,0.0014,0.0029,0.0083,0.7864,11603.0,18723,24500,15500,13080,0.5894,0.6022,0.0028,0.0000,0.0,0.0000,0.0133,0.0004,0.0713,0.0,0.0111,0.0046,0.0154,0.0062,0.0354,0.0079,0.0000,0.0102,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0127,0.0287,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0043,0.3227,0.2650,0.0000,655.0,12329.0,Liberal Arts and Humanities,53878.3275,56419.9476,55781.4792,166079.7543,0.3668,36.6816,1.8652,186.5177,5.3460,534.5983,203.0,83.0,72.0,52.0,29.0
28625,216852.0,Widener University,"Private, nonprofit",PA,19013-5792,Chester,2.0,1.0,3.0,3.0,Bachel

In [12]:
# inspect remaining nulls
remaining_nulls = train_imputed.isnull().sum()

with pd.option_context('display.max_rows', None):
    print(remaining_nulls) # zip codes still missing

unit_id_institution                                   0
college_name                                          0
institution_control                                   0
state_post_code                                       0
zip_code                                              0
city                                                  0
region_ipeds                                          0
title_IV_eligibility                                  0
pred_degree                                           0
pred_degree_0and4                                     0
degree_name                                           0
degree_code                                           0
major_code                                            0
major_name                                            0
branch_number                                         0
avg_net_price_public                                  0
avg_net_price_private                                 0
full_time_net_tuition_revenue                   

In [13]:
# impute missing values in validate and test 

validate_imputed, test_imputed = prepare.impute_val_and_test(train, validate, test)

print(f'validate shape: {validate_imputed.shape}')
print(f'test shape: {test_imputed.shape}')

validate shape: (16451, 112)
test shape: (13710, 112)


In [14]:
validate.head()

,unit_id_institution,college_name,institution_control,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,degree_name,degree_code,major_code,major_name,branch_number,avg_net_price_public,avg_net_price_private,full_time_net_tuition_revenue,off_campus_cost_of_attendace,on_campus_cost_of_attendace,admission_rate,graduate_number,ACT_score_mid,avg_sat_admitted,required_score,avg_faculty_salary,online_only,comp_rt_ft_150over_expected_time,comp_rt_ft_150over_expected_time_two_races,comp_rt_ft_150over_expected_time_native_american,comp_rt_ft_150over_expected_time_asian,comp_rt_ft_150over_expected_time_black,comp_rt_ft_150over_expected_time_hispanic,comp_rt_ft_150over_expected_time_non_resident,comp_rt_ft_150over_expected_time_unknown_race,comp_rt_ft_150over_expected_time_white,share_entering_students_first_ft,share_of_part_time,first_time_ft_student_retention,first_time_pt_student_retention,enrollment_share_two_races,enrollment_share_native_american,enrollment_share_asian,enrollment_share_black,enrollment_share_hispanic,enrollment_share_pac_islander,enrollment_share_non_resident,enrollment_share_unknown,enrollment_share_white,undergraduate_number_pell_grant_fedral_loan,med_debt_pell_students,median_debt_completed,median_debt_non_first_generation,median_debt_non_pell,fedral_loan_full_time_first_time_undergraduate,pell_grant_full_time_first_time_undergraduate,deg_percent_awarded_agriculture_operations,deg_percent_awarded_natural_resources,deg_percent_awarded_architecture,deg_percent_awarded_area_ethnic_cultural_gender,deg_percent_awarded_communication_journalism,deg_percent_awarded_communication_tech,deg_percent_awarded_computer_science,deg_percent_awarded_personal_culinary_services,deg_percent_awarded_education,deg_percent_awarded_engineering,deg_percent_awarded_engineering_tech,deg_percent_awarded_foreign_language_literatures,deg_percent_awarded_human_science,deg_percent_awarded_legal_profession,deg_percent_awarded_english_lang,deg_percent_awarded_general_studies,deg_percent_awarded_library_sciences,deg_percent_awarded_bio_sciences,deg_percent_awarded_mathematics,deg_percent_awarded_military_tech,deg_percent_awarded_intedisciplinary_studies,deg_percent_awarded_leisure_fitness,deg_percent_awarded_philosophy,deg_percent_awarded_theology,deg_percent_awarded_physical_sciences,deg_percent_awarded_science_tech,deg_percent_awarded_psychology,deg_percent_awarded_homeland_security,deg_percent_awarded_public_admin,deg_percent_awarded_social_sciences,deg_percent_awarded_construction_trades,deg_percent_awarded_mechanic_repair,deg_percent_awarded_precision_production,deg_percent_awarded_transportation_materials,deg_percent_awarded_visual_and_performing_arts,deg_percent_awarded_health,deg_percent_awarded_business_management,deg_percent_awarded_history,non_deg_seeking,avg_net_price,major_category,2017,2018,2019,Grand Total,roi_5yr,pct_roi_5yr,roi_10yr,pct_roi_10yr,roi_20yr,pct_roi_20yr,income_0_30000,income_30001_48000,income_48001_75000,income_75001_110000,income_over_110000
33227,198066.0,Brevard College,"Private, nonprofit",NC,28712,Brevard,5.0,1.0,3.0,3.0,Bachelor’s Degree,3.0,4201.0,"Psychology, General.",1.0,0.0,21716.0,13111.0,6750.0,10800.00000,0.4609,2434.161506,24.2762,1166.2046,2.000000,5711.0,0.0,0.3349,0.218800,0.500000,0.666700,0.2414,0.2500,0.300000,0.375000,0.3590,0.8837,0.0158,0.6162,0.517998,0.0587,0.0113,0.0086,0.1246,0.0702,0.0014,0.0086,0.0372,0.6777,799.0,11000,26000,9750,9500,0.7033,0.4146,0.0000,0.0252,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0157,0.0000,0.0000,0.0000,0.0000,0.0000,0.0453,0.0000,0.0,0.0315,0.0271,0.0,0.0591,0.0813,0.0079,0.0,0.0000,0.0,0.0315,0.0822,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0825,0.0709,0.1496,0.0275,10.0,21716.0,Social Sciences,69607.0267,70888.9152,74486.2376,214982.1795,0.5154,51.5372,2.2072,220.7190,6.1374,613.7410,169.0,13.0,13.0,7.0,4.0
56824,156295.0,Berea College,"Private, nonprofit",KY,40404-2182,Berea,5.0,1.0,3.0,3.0,Bachelor’s Degree,3.0,5138.0,"Registered 